In [1]:
import glob
import pandas as pd
from pathlib import Path

In [2]:
import re

### API

In [3]:
import sys
sys.path.append('../../')
from utils.data.transfer import copy_data_mkdir, run_task_multithread

#### Data Infomation

In [4]:
root = Path('/nas/chenyi/datasets_cls/gallery_detect/')
det_root = Path('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/')

In [26]:
show_root = root /'gallery_oss/dadetv4'
!ls $show_root/

crops_manual  @eaDir  gall_dadetv4_all	gall_dadetv4_rev  info


In [47]:
show_root = root /'gallery_oss/dadetv4/crops_manual/skirt/skirt'
!ls $show_root

01  02	0a  0b	0c  0d	0e  0f	1a  1b	1c  1d	1e  1f	2a  2b	2c  2d	2e  2f


In [45]:
show_root = root /'gallery_oss/dadetv4/gall_dadetv4_all/crops4clear/skirt'
!ls $show_root

01  02	0a  0b	0c  0d	0e  0f	1a  1b	1c  1d	1e  1f	2a  2b	2c  2d	2e  2f


In [37]:
img_org = glob.glob(str(root /'gallery_oss/dadetv4/gall_dadetv4_all/crops4clear/bag/*/*/*'))

In [39]:
img_clr = glob.glob(str(root /'gallery_oss/dadetv4/crops_manual/bag/*/*/*'))

In [40]:
len(img_clr)

20014

In [41]:
len(img_clr)/len(img_org)

0.8576448405896469

In [43]:
img_org = glob.glob(str(root /'gallery_oss/dadetv4/gall_dadetv4_all/crops4clear/lowerbody/*/*/*'))
img_clr = glob.glob(str(root /'gallery_oss/dadetv4/crops_manual/lowerbody/*/*/*'))
len(img_clr)/len(img_org)

0.9833566154460187

In [48]:
img_org = glob.glob(str(root /'gallery_oss/dadetv4/gall_dadetv4_all/crops4clear/skirt/*/*/*'))
img_clr = glob.glob(str(root /'gallery_oss/dadetv4/crops_manual/skirt/*/*/*'))
len(img_clr)/len(img_org)

0.9478269193949838

##### Dataload

In [43]:
# imgs_gall_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/csv/gall_df.csv')
# imgs_gall_df.to_csv('/nas/chenyi/datasets_cls/gallery_detect/csv/gall_df.csv', index=False)

In [ ]:
%%time
imgs_rev_list = glob.glob(str(det_root /'gall_dadetv4_rev/imgs_det/*.jpg'))

In [34]:
1083250/1661397

0.6520115300557302

In [35]:
imgs_gall_rev = imgs_gall_df[1083250:]

In [44]:
imgs_gall_df.head()

,oss_sp,stem
0,/nas/oss/gallery/69/31/69313640-af52-465e-88c8...,69313640-af52-465e-88c8-c0a0099ca5c7-699544
1,/nas/oss/gallery/69/31/69314608-5ca4-4891-8a72...,69314608-5ca4-4891-8a72-ac4cbbc7033a-445882
2,/nas/oss/gallery/69/31/6931afe9-4e7a-4c49-8a87...,6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767
3,/nas/oss/gallery/69/31/6931a1dc-76d5-4de9-9827...,6931a1dc-76d5-4de9-9827-3c5b4841fd26-674980
4,/nas/oss/gallery/69/31/6931d292-4a64-4b28-97a0...,6931d292-4a64-4b28-97a0-64ace65c87e4-411370


In [41]:
%%time
imgs_gall_df['stem'] = imgs_gall_df['oss_sp'].apply(lambda x: Path(x).stem)

CPU times: user 4.59 s, sys: 24.3 ms, total: 4.61 s
Wall time: 4.61 s


In [6]:
!ls /nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all

crops  imgs_det  labels


In [9]:
gall = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/csv/gallery00.csv')

In [11]:
gall.head(2).values

array([['/nas/oss/gallery/00/00/000006d9-d433-4d50-80b1-2c88518e9f5e-481515.jpg'],
       ['/nas/oss/gallery/00/00/00004b07-40c8-488a-a408-b7e7faace3c7-819801.jpg']],
      dtype=object)

#### 数据拷贝

In [74]:
import json
with open('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/.json', 'r') as f:
    crops_rev_list = json.load(f)

In [86]:
labels_rev_list = glob.glob('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_rev/labels/*')
len(labels_rev_list)

577959

In [61]:
# labels_rev_list = [p.replace('dadetv4/gall_dadetv4_rev/crops/', 'dadetv4/gall_dadetv4_rev/labels/') for p in crops_rev_list]
# labels_rev_list = [p.replace('.jpg', '.txt') for p in labels_rev_list]

In [88]:
labels_list = [p.replace('dadetv4/gall_dadetv4_rev/labels', 'dadetv4/gall_dadetv4_all/labels') for p in labels_rev_list]

In [89]:
labels_rev_list[0], labels_list[0]

('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_rev/labels/ffa73c89-b27a-4c6e-b377-c12b49ee0aec-441022.txt',
 '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/labels/ffa73c89-b27a-4c6e-b377-c12b49ee0aec-441022.txt')

In [90]:
input_list = [list(item) for item in zip(labels_rev_list, labels_list)]

In [91]:
%%time
import shutil
def copy_data_mkdir(plist, mode='move'):
    ps = Path(plist[0])
    pt = Path(plist[1])
    if not pt.parent.is_dir():
        pt.parent.mkdir(parents=True, exist_ok=True)
    if (not pt.is_file()) and (ps.is_file()):
        if mode=='move':
            Path.rename(ps, pt)
        else:
            shutil.copyfile(ps, pt)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.53 µs


In [92]:
input_list[0]

['/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_rev/labels/ffa73c89-b27a-4c6e-b377-c12b49ee0aec-441022.txt',
 '/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/labels/ffa73c89-b27a-4c6e-b377-c12b49ee0aec-441022.txt']

In [93]:
%%time           
run_task_multithread(input_list, copy_data_mkdir, 20)

CPU times: user 37.9 s, sys: 47.3 s, total: 1min 25s
Wall time: 5min 29s


#### 数据删除

In [19]:
det_root = Path('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4')

In [14]:
len(imgs_gall_df)

1661397

In [23]:
imgs_gall1m_df = imgs_gall_df[:int(1e6)]
imgs_gall_rev_df = imgs_gall_df[int(1e6):]

In [22]:
len(imgs_gall1m_df)

1000000

In [ ]:
%%time
import shutil
def copy_data_mkdir(plist, mode='copy'):
    ps = Path(plist[0])
    pt = Path(plist[1])
    if not pt.parent.is_dir():
        pt.parent.mkdir(parents=True, exist_ok=True)
    if not pt.is_file():
        if mode=='move':
            Path.rename(ps, pt)
        else:
            shutil.copyfile(ps, pt)
            
run_task_multithread(input_list, copy_data_mkdir)

### 分析img和label的数据

In [76]:
imgs_df['stem'] = imgs_df['path'].apply(lambda x: Path(x).stem)

In [77]:
txt_df['stem'] = txt_df['path'].apply(lambda x: Path(x).stem)

In [88]:
imgs_cross = imgs_df[imgs_df['stem'].isin(txt_df['stem'])]

In [91]:
txt_cross = txt_df[txt_df['stem'].isin(imgs_df['stem'])]

In [80]:
imgs_df.sort_values('stem',inplace=True)
txt_df.sort_values('stem',inplace=True)

In [92]:
len(imgs_df),len(txt_df), len(imgs_cross), len(txt_cross)

(478504, 453275, 453275, 453275)

In [93]:
imgs_cross.sort_values('stem',inplace=True)
txt_cross.sort_values('stem',inplace=True)

/tmp/ipykernel_411860/859653631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imgs_cross.sort_values('stem',inplace=True)


In [94]:
watoo_df = pd.DataFrame()
watoo_df['img_p'] = imgs_cross['path'].values
watoo_df['label_p'] = txt_cross['path'].values

#### 未被检测的数据

In [98]:
imgs_nodet = imgs_df[~imgs_df['stem'].isin(txt_df['stem'])]

In [99]:
len(imgs_nodet)

25229

In [134]:
# imgs_nodet.to_csv('./data/watoo470k/imgs_nodet.csv', index=False)

In [119]:
p = imgs_nodet['path'].values[9]

#### 解析labels txt

In [13]:
p = txt_df.head(1).values[0][0]
!cat $p

2 0.505992 0.516117 0.985353 0.78036 0.810297


In [49]:
p = txt_df.head(10).values[-4][0]
!cat $p

3 0.506 0.923333 0.305333 0.153333 0.707341
2 0.464667 0.59 0.572 0.545333 0.958252


In [74]:
with open(p,'r') as f:
    txt = f.readlines()

In [75]:
txt

['3 0.506 0.923333 0.305333 0.153333 0.707341\n',
 '2 0.464667 0.59 0.572 0.545333 0.958252\n']

In [69]:
for item in txt:
    list1 = re.split(r'\n', item)
    list2 = list1[0].split(' ')
    

#### 获取图片宽高

In [13]:
!ls /nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/

crops_all_df.csv	imgs_det_df.csv     rewrite_labels_plist.json
crops_imgs1m_list.json	imgs_rev_list.json
crops_rev_list.json	labels_df.csv


In [30]:
# imgs_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/imgs_det_df.csv')
imgs_df.to_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/imgs_det_df.csv', index=False)

In [15]:
len(imgs_df)

1661397

In [20]:
imgs_df['crops_p'] = imgs_df['crops_p'].apply(lambda x: Path(x))
imgs_df['stem'] = imgs_df['crops_p'].apply(lambda x: x.stem)

In [ ]:
# from tqdm import tqdm
# from PIL import Image
# for p in tqdm(imgs_df['det_p'].values):
#     img = Image.open(p)
#     w, h = img.size
#     w_list.append(w)
#     h_list.append(h)

In [40]:
from PIL import Image
def get_img_hw(p):
    img = Image.open(p)
    w, h = img.size
    w_list.append(w)
    h_list.append(h)

In [43]:
%%time
w_list = []
h_list = []
run_task_multithread(imgs_df['det_p'].values.tolist(), get_img_hw, 20)

CPU times: user 3min 56s, sys: 3min 5s, total: 7min 1s
Wall time: 42min 57s


In [44]:
imgs_df['width'] = w_list
imgs_df['height'] = h_list

In [45]:
imgs_df.to_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/imgs_det_df.csv', 
               index=False)

In [46]:
imgs_df.head()

,det_p,stem,width,height
0,/nas/chenyi/datasets_cls/gallery_detect/galler...,69313640-af52-465e-88c8-c0a0099ca5c7-699544,933,933
1,/nas/chenyi/datasets_cls/gallery_detect/galler...,69314608-5ca4-4891-8a72-ac4cbbc7033a-445882,1080,1441
2,/nas/chenyi/datasets_cls/gallery_detect/galler...,6931afe9-4e7a-4c49-8a87-72630dfbf5d2-345767,450,450
3,/nas/chenyi/datasets_cls/gallery_detect/galler...,6931a1dc-76d5-4de9-9827-3c5b4841fd26-674980,1440,1800
4,/nas/chenyi/datasets_cls/gallery_detect/galler...,6931d292-4a64-4b28-97a0-64ace65c87e4-411370,690,1051


### 画图

In [29]:
!pwd

/home/chenyi/workspace/myPyModule99/gallery_manage


In [31]:
import sys
sys.path.append('../')

In [36]:
from utils.plots.plots import draw_bbox
from utils.bbox.conversation import *
from utils.display.html import *

#### 数据拷贝

In [124]:
import sys
sys.path.append('../')
from utils.data.transfer import copy_data_mkdir, run_task_multithread

#### target path

In [133]:
imgs_nodet.head(2).values

array([[PosixPath('/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/hwpedia_folder/卫衣/150/005a2302-330b-4afc-ae19-5d0e513ee14a.jpg'),
        '005a2302-330b-4afc-ae19-5d0e513ee14a',
        '/nas/lichangjian/wardrobe_2/done_/卫衣/005a2302-330b-4afc-ae19-5d0e513ee14a.jpg'],
       [PosixPath('/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/hwpedia_folder/牛仔裤/80/009dcf1b-1bb6-435e-b30c-0678ada49aee.jpg'),
        '009dcf1b-1bb6-435e-b30c-0678ada49aee',
        '/nas/lichangjian/wardrobe_2/done_/牛仔裤/009dcf1b-1bb6-435e-b30c-0678ada49aee.jpg']],
      dtype=object)

In [177]:
sroot = '/nas/lichangjian/wardrobe_2/done_'
troot = '/nas/chenyi/datasets_cls/gallery_detect/watoo400k/analysis/hwpedia/nodet_kfolder'

In [146]:
!ls /nas/chenyi/datasets_cls/gallery_detect/watoo400k/analysis/hwpedia/

nodet


In [136]:
imgs_nodet['path'] = imgs_nodet['path'].apply(lambda x: Path(x))

/tmp/ipykernel_411860/2584459815.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imgs_nodet['path'] = imgs_nodet['path'].apply(lambda x: Path(x))


In [137]:
imgs_nodet['img_sp'] = imgs_nodet['path'].apply(lambda x: '{0}/{1}/{2}'.format(sroot, x.parts[-3], x.name))

/tmp/ipykernel_411860/1833358304.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imgs_nodet['img_sp'] = imgs_nodet['path'].apply(lambda x: '{0}/{1}/{2}'.format(sroot, x.parts[-3], x.name))


In [178]:
imgs_nodet['img_tp'] = imgs_nodet['path'].apply(lambda x: '{0}/{1}_{2}/{3}'.format(troot, folder2key[x.parts[-3]], x.parts[-3], x.name))

/tmp/ipykernel_411860/361264578.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imgs_nodet['img_tp'] = imgs_nodet['path'].apply(lambda x: '{0}/{1}_{2}/{3}'.format(troot, folder2key[x.parts[-3]], x.parts[-3], x.name))


In [180]:
input_list[0]

['/nas/lichangjian/wardrobe_2/done_/卫衣/005a2302-330b-4afc-ae19-5d0e513ee14a.jpg',
 '/nas/chenyi/datasets_cls/gallery_detect/watoo400k/analysis/hwpedia/nodet_kfolder/10_卫衣/005a2302-330b-4afc-ae19-5d0e513ee14a.jpg']

In [181]:
%%time
import shutil
input_list = [list(item) for item in zip(imgs_nodet['img_sp'].values, imgs_nodet['img_tp'].values)]
def copy_data_mkdir(plist, mode='copy'):
    ps = Path(plist[0])
    pt = Path(plist[1])
    if not pt.parent.is_dir():
        pt.parent.mkdir(parents=True, exist_ok=True)
    if not pt.is_file():
        if mode=='move':
            Path.rename(ps, pt)
        else:
            shutil.copyfile(ps, pt)
            
run_task_multithread(input_list, copy_data_mkdir)

CPU times: user 5.52 s, sys: 31.3 s, total: 36.9 s
Wall time: 37.1 s
